<a href="https://colab.research.google.com/github/aecins/tutorials/blob/main/ml/mnist_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import plotly.express as px
from torch import nn
from tqdm import tqdm

In [2]:
import os

NUM_CPU_CORES = os.cpu_count()
print(f"Number of available CPU cores: {NUM_CPU_CORES}")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")


Number of available CPU cores: 2
Device: cpu


In [3]:
# Create datasets
dataset_train = torchvision.datasets.MNIST(
    root = './data',
    train = True,
    transform = torchvision.transforms.ToTensor(),
    download = True,
)

dataset_test = torchvision.datasets.MNIST(
    root = './data',
    transform = torchvision.transforms.ToTensor(),
    train = False,
)

print("\n")
print(f"Train dataset size: {len(dataset_train)}")
print(f"Test dataset size: {len(dataset_test)}")

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 508kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.60MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.76MB/s]



Train dataset size: 60000
Test dataset size: 10000


In [4]:
image, label = dataset_train[213]
fig = px.imshow(image.squeeze())
fig.show()
print(f"Label: {label}")
print(f"Image size: {image.shape}")
print(f"Image type: {image.type()}")

Label: 2
Image size: torch.Size([1, 28, 28])
Image type: torch.FloatTensor


In [5]:
BATCH_SIZE = 100

# Create dataloaders
dataloader_train = torch.utils.data.DataLoader(
    dataset=dataset_train,
    batch_size=BATCH_SIZE,
    num_workers=NUM_CPU_CORES,
    shuffle=True,
    drop_last=True,
    pin_memory=True if device == torch.device("cuda") else False,
)

dataloader_test = torch.utils.data.DataLoader(
    dataset=dataset_test,
    batch_size=BATCH_SIZE,
    num_workers=NUM_CPU_CORES,
    shuffle=False,
    drop_last=True,
    pin_memory=True if device == torch.device("cuda") else False,
)

In [6]:
images_batch, labels_batch = next(iter(dataloader_train))
print(f"Batch size: {images_batch.shape[0]}")
print(f"Batch image tensor size: {images_batch.shape}")
print(f"Batch label tensor size: {labels_batch.shape}")

Batch size: 100
Batch image tensor size: torch.Size([100, 1, 28, 28])
Batch label tensor size: torch.Size([100])


In [7]:
# Create a classifier network
class MnistClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=hidden_size, out_features=num_classes, bias=True)

    def forward(self, x):
        # x (batch_size, 1, height, width)
        x = x.view(x.size(0), -1)   # (batch_size, height*width)
        x = self.fc1(x)             # (batch_size, hidden_size)
        x = self.relu(x)            # (batch_size, hidden_size)
        x = self.fc2(x)             # (batch_size, 10)

        return x


In [16]:
model = MnistClassifier(
    input_size=28*28,
    hidden_size=128,
    num_classes=10,
).to(device)

num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"The model has {num_trainable_params:,} trainable parameters.")

The model has 101,770 trainable parameters.


In [17]:
# Create loss
criterion = nn.CrossEntropyLoss(reduction="mean")

In [18]:
# Create optmizer
LEARNING_RATE = 5e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [19]:
# Create training loop
NUM_EPOCHS = 100

model.train()

for epoch_idx in range(NUM_EPOCHS):
  epoch_loss = 0
  for batch_idx, (batch_images, batch_labels) in enumerate(dataloader_train):
    batch_images = batch_images.to(device)
    batch_labels = batch_labels.to(device)

    optimizer.zero_grad()

    prediction = model(batch_images)
    loss = criterion(prediction, batch_labels)

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item() * batch_images.size(0)

  epoch_loss /= len(dataloader_train)

  print(f"Epoch: {epoch_idx:>6}, Loss: {epoch_loss:.8f}, lr: {LEARNING_RATE:.8f}")

Epoch:      0, Loss: 50.02324101, lr: 0.00050000
Epoch:      1, Loss: 23.78081670, lr: 0.00050000
Epoch:      2, Loss: 18.44084544, lr: 0.00050000
Epoch:      3, Loss: 14.97850780, lr: 0.00050000
Epoch:      4, Loss: 12.59596735, lr: 0.00050000
Epoch:      5, Loss: 10.75785958, lr: 0.00050000
Epoch:      6, Loss: 9.35339700, lr: 0.00050000
Epoch:      7, Loss: 8.18273931, lr: 0.00050000
Epoch:      8, Loss: 7.21168239, lr: 0.00050000
Epoch:      9, Loss: 6.39683364, lr: 0.00050000
Epoch:     10, Loss: 5.72481742, lr: 0.00050000
Epoch:     11, Loss: 5.09611895, lr: 0.00050000
Epoch:     12, Loss: 4.65098176, lr: 0.00050000
Epoch:     13, Loss: 4.15809685, lr: 0.00050000
Epoch:     14, Loss: 3.74752043, lr: 0.00050000
Epoch:     15, Loss: 3.36944968, lr: 0.00050000
Epoch:     16, Loss: 3.02251079, lr: 0.00050000
Epoch:     17, Loss: 2.75079413, lr: 0.00050000
Epoch:     18, Loss: 2.47734538, lr: 0.00050000
Epoch:     19, Loss: 2.19151011, lr: 0.00050000
Epoch:     20, Loss: 1.98052280, l

In [20]:
# Test model
model.eval()

num_correct = 0

for batch_idx, (batch_images, batch_gt_labels) in enumerate(tqdm(dataloader_test)):
  batch_images = batch_images.to(device)
  batch_gt_labels = batch_gt_labels.to(device)

  prediction = model(batch_images)

  _, batch_predicted_labels = torch.max(prediction, 1)
  batch_prediction_error = batch_predicted_labels - batch_gt_labels

  batch_num_correct = torch.sum(batch_prediction_error == 0)
  num_correct += batch_num_correct

accuracy = num_correct / len(dataset_test)
print(f"Accuracy: {accuracy:.8f}")

100%|██████████| 100/100 [00:01<00:00, 55.29it/s]

Accuracy: 0.97920001
